# **🏠 지하철 좌표 및 유동인구**
> 아파트와 가장 가까운 지하철 역에 대한 거리를 계산하여 그 역이 역세권인지 확인
> 해당 역의 유동인구 가져오기
> 서울의 역세권 기준
> - 초역세권 : 지하철역과 아파트와의 거리가 250m 이내
> - 역세권 : 지하철역과 아파트와의 거리가 500m 이내

## Contents
- Library Import
- Data Load
- Haversin Distance Calculate

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance
import time

## 2. Data Load

### 2.1. 데이터 로드

- 지하철 좌표와 아파트 좌표를 하버사인 거리로 계산해서 유동인구, 역세권 유무도 표시

In [2]:
# 데이터 불러오기
train = pd.read_csv('../data/train_latlon_complete.csv')
subway_people = pd.read_csv('../data/subway.csv', encoding='cp949')
subway_latlon = pd.read_csv('../data/서울시 역사마스터 정보.csv', encoding='cp949')

In [38]:
# 위경도 좌표
subway_latlon

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
766,154,종로5가,1호선,37.570926,127.001849
767,153,종로3가,1호선,37.570406,126.991847
768,152,종각,1호선,37.570161,126.982923
769,151,시청,1호선,37.565715,126.977088


In [53]:
# 혹시나 없는 노선 체크 : 신분당선이 없는 것을 늦게 확인함
subway_people['호선명'].unique()

array(['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선',
       '9호선2~3단계', '경강선', '경부선', '경원선', '경의선', '경인선', '경춘선', '공항철도 1호선',
       '과천선', '분당선', '서해선', '수인선', '신림선', '안산선', '우이신설선', '일산선', '장항선',
       '중앙선', '9호선2단계'], dtype=object)

In [4]:
subway_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63725 entries, 0 to 63724
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   사용월           63725 non-null  int64 
 1   호선명           63725 non-null  object
 2   지하철역          63725 non-null  object
 3   04시-05시 승차인원  63725 non-null  int64 
 4   04시-05시 하차인원  63725 non-null  int64 
 5   05시-06시 승차인원  63725 non-null  int64 
 6   05시-06시 하차인원  63725 non-null  int64 
 7   06시-07시 승차인원  63725 non-null  int64 
 8   06시-07시 하차인원  63725 non-null  int64 
 9   07시-08시 승차인원  63725 non-null  int64 
 10  07시-08시 하차인원  63725 non-null  int64 
 11  08시-09시 승차인원  63725 non-null  int64 
 12  08시-09시 하차인원  63725 non-null  int64 
 13  09시-10시 승차인원  63725 non-null  int64 
 14  09시-10시 하차인원  63725 non-null  int64 
 15  10시-11시 승차인원  63725 non-null  int64 
 16  10시-11시 하차인원  63725 non-null  int64 
 17  11시-12시 승차인원  63725 non-null  int64 
 18  11시-12시 하차인원  63725 non-null  int64 
 19  12시-

In [5]:
subway_people.columns

Index(['사용월', '호선명', '지하철역', '04시-05시 승차인원', '04시-05시 하차인원', '05시-06시 승차인원',
       '05시-06시 하차인원', '06시-07시 승차인원', '06시-07시 하차인원', '07시-08시 승차인원',
       '07시-08시 하차인원', '08시-09시 승차인원', '08시-09시 하차인원', '09시-10시 승차인원',
       '09시-10시 하차인원', '10시-11시 승차인원', '10시-11시 하차인원', '11시-12시 승차인원',
       '11시-12시 하차인원', '12시-13시 승차인원', '12시-13시 하차인원', '13시-14시 승차인원',
       '13시-14시 하차인원', '14시-15시 승차인원', '14시-15시 하차인원', '15시-16시 승차인원',
       '15시-16시 하차인원', '16시-17시 승차인원', '16시-17시 하차인원', '17시-18시 승차인원',
       '17시-18시 하차인원', '18시-19시 승차인원', '18시-19시 하차인원', '19시-20시 승차인원',
       '19시-20시 하차인원', '20시-21시 승차인원', '20시-21시 하차인원', '21시-22시 승차인원',
       '21시-22시 하차인원', '22시-23시 승차인원', '22시-23시 하차인원', '23시-24시 승차인원',
       '23시-24시 하차인원', '00시-01시 승차인원', '00시-01시 하차인원', '01시-02시 승차인원',
       '01시-02시 하차인원', '02시-03시 승차인원', '02시-03시 하차인원', '03시-04시 승차인원',
       '03시-04시 하차인원', '작업일자'],
      dtype='object')

- 더하는 것이 어려워서 일일이 값을 대입해서 더하기

In [6]:
# 하루 유동인구 합 구하기
subway_people['유동승객수'] = subway_people[['04시-05시 승차인원', '04시-05시 하차인원', '05시-06시 승차인원',
       '05시-06시 하차인원', '06시-07시 승차인원', '06시-07시 하차인원', '07시-08시 승차인원',
       '07시-08시 하차인원', '08시-09시 승차인원', '08시-09시 하차인원', '09시-10시 승차인원',
       '09시-10시 하차인원', '10시-11시 승차인원', '10시-11시 하차인원', '11시-12시 승차인원',
       '11시-12시 하차인원', '12시-13시 승차인원', '12시-13시 하차인원', '13시-14시 승차인원',
       '13시-14시 하차인원', '14시-15시 승차인원', '14시-15시 하차인원', '15시-16시 승차인원',
       '15시-16시 하차인원', '16시-17시 승차인원', '16시-17시 하차인원', '17시-18시 승차인원',
       '17시-18시 하차인원', '18시-19시 승차인원', '18시-19시 하차인원', '19시-20시 승차인원',
       '19시-20시 하차인원', '20시-21시 승차인원', '20시-21시 하차인원', '21시-22시 승차인원',
       '21시-22시 하차인원', '22시-23시 승차인원', '22시-23시 하차인원', '23시-24시 승차인원',
       '23시-24시 하차인원', '00시-01시 승차인원', '00시-01시 하차인원', '01시-02시 승차인원',
       '01시-02시 하차인원', '02시-03시 승차인원', '02시-03시 하차인원', '03시-04시 승차인원',
       '03시-04시 하차인원']].sum(axis = 1)

- 2022년 ~ 2023년 유동인구는 값을 제대로 입력받지 못한 역이 있음 : 까치울역 ...
- 공사중과 같은 이유로 집계되지 않았을 수 있음
- 따라서 2021년 유동인구를 대체하기로함 
- 코로나 시즌이라 정보가 부정확할 수 있음

In [7]:
# 2021년 유동인구 가져오기
subway_people2 = subway_people[(subway_people['사용월'] > 202013) & (subway_people['사용월'] <= 202200)][['지하철역', '유동승객수']]

In [8]:
# 지하철역별 유동인구 계산
t = subway_people2.groupby('지하철역').sum().reset_index()

In [9]:
# (승차 + 하차) * 12
t['월평균_유동승객수'] = t['유동승객수'] // 24 

In [10]:
# 필요한 행만 남기기
subway_needs = t.drop(['유동승객수'], axis = 1)
subway_needs.columns = ['지하철역', '유동승객수']

In [11]:
# 이름 다른 6개 일일이 채워주기
subway_needs['지하철역'][subway_needs['지하철역'] == '낙성대(강감찬)'] = '낙성대'
subway_needs['지하철역'][subway_needs['지하철역'] == '동대문역사문화공원(DDP)'] = '동대문역사문화공원'
subway_needs['지하철역'][subway_needs['지하철역'] == '마곡나루(서울식물원)'] = '마곡나루'
subway_needs['지하철역'][subway_needs['지하철역'] == '용마산(용마폭포공원)'] = '용마산'
subway_needs['지하철역'][subway_needs['지하철역'] == '평택지제'] = '지제'
subway_needs['지하철역'][subway_needs['지하철역'] == '하남시청(덕풍?신장)'] = '하남시청(덕풍-신장)'

## 3. Haversin Distance
- 데이터가 약 9000 x 800 개이므로 cross join과 numpy를 활용하여 계산
- for문 보다 더 빠른 계산을 적용할 수 있음

In [12]:
# 결합
sub = pd.merge(subway_needs, subway_latlon.drop_duplicates('역사명'), how = 'left', left_on = '지하철역', right_on = '역사명')

In [13]:
# 결측확인 및 대입
# sub.loc[331, :] = [1296, 3.0, 1296, '계양역', '경의선', 37.57156, 126.7364]

In [14]:
# 필요한 것만 뽑기
sub_needs = sub[['역사명', '위도', '경도', '유동승객수']]

In [15]:
sub_needs

,역사명,위도,경도,유동승객수
0,4.19민주묘지,37.649502,127.013684,73121
1,가능,37.748577,127.044213,166245
2,가락시장,37.492888,127.118398,420949
3,가산디지털단지,37.480338,126.882656,1516694
4,가양,37.561391,126.854456,498709
...,...,...,...,...
518,회기,37.589460,127.057583,606220
519,회룡,37.725006,127.047073,317384
520,회현(남대문시장),37.558514,126.978246,589452
521,효창공원앞,37.539233,126.961384,239801


In [16]:
# 지번주소, 위도, 경도 가져오기
address = pd.read_csv('../data/address_to_latlon_complete.csv')

In [17]:
# 오류가 있는지 샘플 뽑아 확인
address[address['지번주소'] == '서울특별시 영등포구 영등포동7가 94-99']

,지번주소,좌표X뉴,좌표Y뉴
8136,서울특별시 영등포구 영등포동7가 94-99,126.909869,37.524135


In [18]:
# 본격적 하버사인 거리 구하기
veryhard = pd.merge(address, sub_needs, how='cross')

In [19]:
# 하버사인 거리
lat1 = np.deg2rad(veryhard['경도'])
lat2 = np.deg2rad(veryhard['좌표X뉴'])
lon1 = np.deg2rad(veryhard['위도'])
lon2 = np.deg2rad(veryhard['좌표Y뉴'])
dlat = lat2 - lat1
dlon = lon1 - lon2
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [20]:
# 미터단위 거리 구하기
veryhard['역까지_거리'] = c * 6371 * 1000

In [35]:
# 거리자르기
min_value = veryhard.loc[veryhard.groupby('지번주소')['역까지_거리'].idxmin()]

In [24]:
# 역세권 변수 만들기
min_value['1차역세권'] = np.where(min_value['역까지_거리'] <= 250, 1, 0)
min_value['2차역세권'] = np.where((min_value['역까지_거리'] > 250) & (min_value['역까지_거리'] <= 500), 1, 0)

- 나중에 한번에 붙이기 위해 저장

In [25]:
# 저장
min_value.to_csv('../data/지번주소별_유동인구_역세권_역까지거리.csv', index=False)